In [165]:
from google import genai
from dotenv import load_dotenv
import os
from pydantic import BaseModel, ValidationError
from typing import List
from google.genai.types import GenerateContentConfig, Tool
from google.generativeai.types import HarmCategory, HarmBlockThreshold
import json
import re
import random as rd

# Load environment variables from the .env file
load_dotenv()

# Retrieve the API key from the .env file
api_key = os.getenv('GEMINI_API_KEY')

# Initialize the Google AI client
client = genai.Client(api_key=api_key)

# Generate content
response = client.models.generate_content(
    model='gemini-2.0-flash-exp', contents='What is your name?'
)
print(response.text)




I am a large language model, trained by Google.



*Function to extract Json of LLm output and save it to a file*

In [166]:
def save_json_from_string(input_string, output_filename):
    """
    Extracts JSON content from a string and saves it to a JSON file with the specified name.

    :param input_string: The input string containing JSON data within brackets.
    :param output_filename: The name of the output JSON file (e.g., "data.json").
    :return: True if the operation was successful, False otherwise.
    """
    # Step 1: Extract the JSON content within the brackets
    result = re.search(r'\[.*\]', input_string, re.DOTALL)

    if result:
        extracted_content = result.group(0)  # Get the matched content

        try:
            # Step 2: Parse the extracted content into a Python object (list/dict)
            json_data = json.loads(extracted_content)

            # Step 3: Write the JSON data to a file
            with open(output_filename, "w", encoding="utf-8") as json_file:
                json.dump(json_data, json_file, indent=4)  # Save with pretty formatting

            print(f"JSON data has been successfully written to '{output_filename}'.")
            return True  # Operation was successful
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON: {e}")
        except Exception as e:
            print(f"An error occurred while writing the file: {e}")
    else:
        print("No content found within brackets.")

    return False  # Operation failed

In [167]:
MODEL = "gemini-2.0-flash-exp"

In [168]:

COMPANY = 'Proximus'

In [169]:
safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
]

**SETTING UP THE RESEARCH ASSISTANT** (temperature = 0, tools = google search)

In [170]:
system_instruction = """You are an analyst that conducts company research.
You are given a company name, and you will work on a company report. You have access
to Google Search to look up company news, updates, metrics, public records and linkedin pages to write research reports.

When given a company name, identify key aspects to research, look up that information
and then write an elaborate company report. 

Thoroughly plan your work in detail and steps, but avoid discussing it. Do not add any additional comments after finishing the report."""

In [171]:
config = GenerateContentConfig(system_instruction=system_instruction, tools=[Tool(google_search={})], temperature=0,maxOutputTokens=8000, top_p=0.9, top_k=5, safety_settings=safety_settings)

**Main Company report**

In [172]:
contents = f"""
Write a report about {COMPANY}.

Add to the report a link to the most recent company logo, the company colors in rgb, a summary of companies who offer services to them, a comprehensive summary of the key employees and their roles in the company, a summary of departments of the company. 

Feel free to plan your work and talk about it, but when you start writing the report,
put a line of dashes (---) to demarcate the report itself, and say nothing else after
the report has finished.
"""

response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)

print(response.text)


Okay, I will create a report about Proximus. Here's my plan:

1.  **Company Overview:** I'll start by gathering general information about Proximus, including its industry, history, and overall mission.
2.  **Logo and Colors:** I'll search for the most recent company logo and identify its official colors, expressing them in RGB format.
3.  **Service Providers:** I'll research companies that provide services to Proximus, focusing on key partnerships and suppliers.
4.  **Key Employees:** I'll identify key employees, their roles, and their responsibilities within the company. I will use LinkedIn to find this information.
5.  **Company Departments:** I'll outline the main departments within Proximus and their functions.
6.  **Report Compilation:** Finally, I'll compile all the gathered information into a comprehensive report.

Now, let's begin the research.

---
# Proximus Company Report

**Company Overview:**

Proximus is a Belgian telecommunications company that provides a wide range of s

**Proximus Key Employees and Roles**

In [173]:
contents = f"""
Write a report about {COMPANY}

The report should only contain a comprehensive summary of the employees in the company. 

The report should contain, the complete first name and family name, an email address with this structure "first name.family name@{COMPANY}.be", the role that they have within the company and the department that they make part of.

Do not include abbreviations in names, and exclude employees whose full first and last names cannot be found.

Give me as much grounded names that you can find with your research and put them all in the report.

The report should be written in this style example:

Use this JSON schema:

Proximus_employees = {{'first_name':str, 'family_name':str, 'role':str, 'department':str, 'email_address':str}}
Return: list[Proximus_employees]
"""

response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)

print(response.text)

```json
[
    {
        "first_name": "Guillaume",
        "family_name": "Boutin",
        "role": "Chief Executive Officer",
        "department": "CEO's department",
        "email_address": "guillaume.boutin@Proximus.be"
    },
    {
        "first_name": "Ben",
        "family_name": "Appel",
        "role": "Corporate Affairs Lead",
        "department": "Corporate Affairs",
        "email_address": "ben.appel@Proximus.be"
    },
     {
        "first_name": "Jim",
        "family_name": "Casteele",
        "role": "Consumer Market Lead",
        "department": "Consumer Market",
        "email_address": "jim.casteele@Proximus.be"
    },
    {
        "first_name": "Anne-Sophie",
        "family_name": "Lotgering",
        "role": "Enterprise IT Services & Segments Lead",
         "department":"Enterprise Market",
        "email_address": "anne-sophie.lotgering@Proximus.be"
    },
    {
        "first_name": "Antonietta",
        "family_name": "Mastroianni",
        "role": "Digi

In [174]:
save_json_from_string(response.text, 'proximus_employees.json')

JSON data has been successfully written to 'proximus_employees.json'.


True

**Departments Proximus**

In [175]:
contents = f"""

Write a report about {COMPANY}

The report should exclusively provide a detailed summary of the company's departments and their respective subdivisions, if any.

Do not include abbreviations in the report.

Provide a detailed list, specifying only verified and public information. Do not include speculative or incomplete details.

The report should be written in this style example:

Use this JSON schema:

Proximus_departments = {{'department':str, 'subdivision':list[str]}}
Return: list[Proximus_departments]
"""

response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)

print(response.text)

```json
[
    {
        "department": "CEO's Department",
        "subdivision": []
    },
    {
        "department": "Consumer Market",
        "subdivision": []
    },
     {
        "department": "Enterprise Market",
        "subdivision": [
            "Proximus NXT"
        ]
    },
    {
        "department": "Corporate Affairs",
         "subdivision": []
    },
    {
        "department": "Network & Wholesale",
        "subdivision": []
    },
    {
        "department":"Customer Operations",
        "subdivision": []
    },
    {
        "department": "Human Capital",
        "subdivision": []
    },
    {
        "department": "Digital & IT",
        "subdivision": []
    },
    {
        "department": "Finance",
        "subdivision": []
    },
    {
        "department": "Proximus Global",
        "subdivision": [
            "BICS",
            "Telesign",
             "Route Mobile"
        ]
    },
     {
        "department": "Proximus Media House",
        "subdivisio

In [176]:
save_json_from_string(response.text, 'proximus_departments.json')

JSON data has been successfully written to 'proximus_departments.json'.


True

**Company Colors**

In [177]:
contents = f"""

Write a report about {COMPANY}

The report should exclusively provide the main colors of Company's branding, including any official color codes such as HEX or RGB. Focus on the primary colors used in the company's logo and website.

Do not include speculative colors and focus only on the main ones.

The report should be written in this style example:

Use this JSON schema:

company_colors = {{'color_name_1':str, 'hex_code_1':str, 'rgb_code_1':list[int]}},{{'color_name_2':str,'hex_code_2':str,'rgb_code_2':list[int]}}
Return: list[company_colors]

Thoroughly plan your work in detail and steps, but avoid discussing it. Do not add any additional comments after finishing the report.
"""


response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)

print(response.text)

```json
[
    {
        "color_name_1": "Primary Purple",
        "hex_code_1": "#5C2D91",
        "rgb_code_1": [92, 45, 145]
    },
    {
        "color_name_2": "Black",
        "hex_code_2": "#000000",
        "rgb_code_2": [0, 0, 0]
    },
    {
        "color_name_3": "White",
        "hex_code_3": "#FFFFFF",
        "rgb_code_3": [255, 255, 255]
    }
]
```


In [178]:
save_json_from_string(response.text, 'proximus_colors.json')

JSON data has been successfully written to 'proximus_colors.json'.


True

**Service providers**

In [179]:
contents = f"""

Write a report about {COMPANY}

List the main service providers of the company. Include any known suppliers, contractors, technology service providers, or any other third-party companies that the company relies on. 
Provide the names of the service providers, type and description of the services they provide to the company. Also include the providers homepage in the report, if you can't find it, don not include the provider in the report. 

The report should be written in this style example:

Use this JSON schema:

company_service = {{"provider":str,"service":str,"type":str,"provider_homepage":str]}}
Return: list[company_service]
"""

response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)

print(response.text)

```json
[
    {
        "provider": "Scarlet",
        "service": "Telecommunication services",
        "type": "Subsidiary company",
        "provider_homepage": "https://www.scarlet.be/"
    },
    {
        "provider": "Mobile Vikings",
        "service": "Telecommunication services",
        "type": "Brand",
         "provider_homepage": "https://mobilevikings.be/en/"
    },
    {
        "provider": "Tango",
        "service": "Telecommunication services",
        "type": "Brand",
        "provider_homepage": "https://www.tango.lu/"
    },
     {
        "provider": "Telindus Luxembourg",
        "service": "Telecommunication and ICT services",
        "type": "Brand",
        "provider_homepage": "https://www.telindus.lu/"
    },
    {
        "provider": "Telindus Netherlands",
         "service": "Telecommunication and ICT services",
        "type": "Brand",
        "provider_homepage": "https://www.telindus.nl/"
    },
    {
        "provider": "BICS",
        "service": "Inte

In [180]:
save_json_from_string(response.text, 'proximus_providers.json')

JSON data has been successfully written to 'proximus_providers.json'.


True

**Providers info**

In [181]:
# Load JSON data from the file
with open("proximus_providers.json", "r") as file:
    providers = json.load(file)

In [182]:
# Function to pick a random provider
def pick_random_provider(providers):
    if not providers:
        return None  # Handle the case where the list is empty
    return rd.choice(providers)

In [183]:
provider = pick_random_provider(providers)

print(provider)

{'provider': 'Ads & Data', 'service': 'Premium media channels and data for advertisers', 'type': 'Joint venture', 'provider_homepage': None}


In [184]:
provider = [provider]
file_path = 'provider_general_data.json'

# Write the data to the JSON file
with open(file_path, 'w') as json_file:
    json.dump(provider, json_file, indent=4)

print(f'Data saved to {file_path}')

Data saved to provider_general_data.json


**Provider Key Employees and Roles**

In [185]:
contents = f"""
    Write a report about company {provider}. This company is a provider of {COMPANY}

    The report should only contain a comprehensive summary of the employees in the company.

    The report should contain, the complete first name and family name, an email address with this structure "first name.family name@{provider}.be", the role that they have within the company and the department that they make part of.

    Do not include abbreviations in names, and exclude employees whose full first and last names cannot be found. Do not include employees of {COMPANY} in the report.

    Give me as much grounded names that you can find with your research and put them all in the report.

    The report should be written in this style example:

    Use this JSON schema:

    Employees = {{'provider': str, 'first_name': str, 'family_name': str, 'role': str, 'department': str, 'email_address':str}}
    Return: list[Employees]
    """
    
response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)

print(response.text)
    

```json
[
    {
        "provider": "Ads & Data",
        "first_name": "Bart",
        "family_name": "Decoster",
        "role": "CEO",
        "department": "Leadership",
        "email_address": "bart.decoster@adsanddata.be"
    },
    {
        "provider": "Ads & Data",
        "first_name": "Bart",
        "family_name": "Demeulenaere",
        "role": "CCO",
         "department": "Leadership",
        "email_address": "bart.demeulenaere@adsanddata.be"
    },
    {
        "provider": "Ads & Data",
        "first_name": "Koen",
        "family_name": "Van Rhijn",
        "role": "COO",
         "department": "Leadership",
        "email_address": "koen.van rhijn@adsanddata.be"
    }
]
```


In [186]:
save_json_from_string(response.text, 'provider_employees.json')

JSON data has been successfully written to 'provider_employees.json'.


True

**Provider Colors**

In [187]:

contents = f"""

    Write a report about company {provider}. This company is a provider of {COMPANY}

    The report should exclusively provide the main colors of Company's branding, including any official color codes such as HEX or RGB. Focus on the primary colors used in the company's logo and website.

    Do not include speculative colors and focus only on the main ones.

    The report should be written in this style example:

    Use this JSON schema:

    company_colors = {{'provider':str,'color_name_1':str, 'hex_code_1':str, 'rgb_code_1':list[int]}},{{'color_name_2':str,'hex_code_2':str,'rgb_code_2':list[int]}}
    Return: list[company_colors]

    Thoroughly plan your work in detail and steps, but avoid discussing it. Do not add any additional comments after finishing the report.
    """


response = client.models.generate_content(
        model=MODEL, config=config, contents=contents)

print(response.text)

```json
[
  {
    "provider": "Ads & Data",
    "color_name_1": "Proximus Purple",
    "hex_code_1": "#5C2D91",
    "rgb_code_1": [92, 45, 145],
    "color_name_2": "White",
    "hex_code_2": "#FFFFFF",
    "rgb_code_2": [255, 255, 255]
  },
  {
    "color_name_1": "Black",
    "hex_code_1": "#000000",
    "rgb_code_1": [0, 0, 0],
    "color_name_2": "Turquoise",
    "hex_code_2": "#66D2CC",
    "rgb_code_2": [102, 210, 204]
  },
    {
    "color_name_1": "Grey",
    "hex_code_1": "#C1C1C3",
    "rgb_code_1": [193, 193, 195],
     "color_name_2": "Magenta",
    "hex_code_2": "#FF418C",
    "rgb_code_2": [255, 65, 140]
  },
  {
    "color_name_1": "Promo Red Positive",
    "hex_code_1": "#DE2A56",
    "rgb_code_1": null,
    "color_name_2": "Promo Red Negative",
    "hex_code_2": "#FF4371",
    "rgb_code_2": null
  }
]
```


In [188]:
save_json_from_string(response.text, 'provider_color.json')

JSON data has been successfully written to 'provider_color.json'.


True

**Departments Provider**

In [189]:
contents = f"""

Write a report about company {provider}. This company is a provider of {COMPANY}

The report should exclusively provide a detailed summary of the company's departments and their respective subdivisions, if any.

Do not include abbreviations in the report.

Provide a detailed list, specifying only verified and public information. Do not include speculative or incomplete details.

The report should be written in this style example:

Use this JSON schema:

Provider_departments = {{'provider':str,'department':str, 'subdivision':list[str]}}
Return: list[Provider_departments]
"""

response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)

print(response.text)

```json
[
  {
    "provider": "Ads & Data",
    "department": "Management",
    "subdivision": [
      "Chief Executive Officer (CEO)",
      "Chief Commercial Officer (CCO)",
      "Chief Operating Officer (COO)"
     ]
  },
  {
    "provider": "Ads & Data",
    "department": "Commercial Teams",
    "subdivision": [
      "Mediahuis Commercial Teams",
      "SBS Belgium Commercial Teams",
       "Pebble Media Commercial Teams"
    ]
  }
]
```


In [190]:
save_json_from_string(response.text, 'provider_departments.json')

JSON data has been successfully written to 'provider_departments.json'.


True

**Blend Data from Json files**

In [191]:
import json
import os

# Function to read JSON data from a file and debug if necessary
def read_json_file(file_path):
    with open(file_path, 'r') as file:
        try:
            data = json.load(file)
            # Debugging: Print the entire content of the file to inspect the structure
            print(f"Data loaded from {file_path} (Type: {type(data)}): {data}")
            if not isinstance(data, list):
                raise ValueError(f"Data in {file_path} is not a list as expected.")
            return data
        except json.JSONDecodeError:
            print(f"Error decoding JSON from {file_path}")
            return []  # Return an empty list if the JSON is malformed

# Function to blend data
def blend_data_from_files(general_file, employee_file, color_file, department_file):
    # Read data from files
    general_data = read_json_file(general_file)
    employee_data = read_json_file(employee_file)
    color_data = read_json_file(color_file)
    department_data = read_json_file(department_file)
    
    # Merge data
    merged_data = {}

    # Add general data
    for entry in general_data:
        try:
            provider = entry["provider"]
            merged_data[provider] = {"general": entry}
        except KeyError:
            print("KeyError: Missing 'provider' in general data entry.")
            continue

    # Add employee data
    for entry in employee_data:
        try:
            provider = entry["provider"]
            if provider in merged_data:
                if "employee" not in merged_data[provider]:
                    merged_data[provider]["employee"] = []
                merged_data[provider]["employee"].append(entry)
        except KeyError:
            print("KeyError: Missing 'provider' in employee data entry.")
            continue

    # Add color data
    for entry in color_data:
        try:
            provider = entry.get("provider", "Unknown")
            if provider in merged_data:
                if "color" not in merged_data[provider]:
                    merged_data[provider]["color"] = {}
                merged_data[provider]["color"].update(entry)
        except KeyError:
            print("KeyError: Missing 'provider' in color data entry.")
            continue

    # Add department data
    for entry in department_data:
        try:
            provider = entry["provider"]
            if provider in merged_data:
                if "departments" not in merged_data[provider]:
                    merged_data[provider]["departments"] = []
                merged_data[provider]["departments"].append(entry)
        except KeyError:
            print("KeyError: Missing 'provider' in department data entry.")
            continue

    return merged_data

# File paths
general_file = 'provider_general_data.json'
employee_file = 'provider_employees.json'
color_file = 'provider_color.json'
department_file = 'provider_departments.json'

# Debugging: Print current working directory
print("Current working directory:", os.getcwd())

# Blend the data
blended_result = blend_data_from_files(general_file, employee_file, color_file, department_file)

# Debugging: Print the blended data to verify
print("Blended Data:", json.dumps(blended_result, indent=4))

# Write the result to a new JSON file and capture any potential errors
try:
    with open('blended_data_providers.json', 'w') as output_file:
        json.dump(blended_result, output_file, indent=4)
    print("Blended data has been saved to 'blended_data_providers.json'")
except Exception as e:
    print(f"Error saving file: {e}")






Current working directory: c:\Users\Rik\Desktop\Proximus-Case-Team-4\Rik
Data loaded from provider_general_data.json (Type: <class 'list'>): [{'provider': 'Ads & Data', 'service': 'Premium media channels and data for advertisers', 'type': 'Joint venture', 'provider_homepage': None}]
Data loaded from provider_employees.json (Type: <class 'list'>): [{'provider': 'Ads & Data', 'first_name': 'Bart', 'family_name': 'Decoster', 'role': 'CEO', 'department': 'Leadership', 'email_address': 'bart.decoster@adsanddata.be'}, {'provider': 'Ads & Data', 'first_name': 'Bart', 'family_name': 'Demeulenaere', 'role': 'CCO', 'department': 'Leadership', 'email_address': 'bart.demeulenaere@adsanddata.be'}, {'provider': 'Ads & Data', 'first_name': 'Koen', 'family_name': 'Van Rhijn', 'role': 'COO', 'department': 'Leadership', 'email_address': 'koen.van rhijn@adsanddata.be'}]
Data loaded from provider_color.json (Type: <class 'list'>): [{'provider': 'Ads & Data', 'color_name_1': 'Proximus Purple', 'hex_code_1'

**SETTING UP THE EMAIL WRITING ASSISTANT** (temperature = 1, tools= ??)

In [192]:
system_instruction= "You are an expert assistant specializing in writing professional, polite, and clear emails. Generate emails based on user input, ensuring proper grammar, tone, and structure."

In [193]:
config = GenerateContentConfig(system_instruction=system_instruction, temperature=1,maxOutputTokens=600, top_p=0.9, top_k=40, safety_settings=safety_settings)

**BLENDING DATA IN HTML**

**SENDING MAILS TO GOPHISH**